### data extraction test
For dynamic brain workshop... testing ways to query Allen data structures for easy analysis.

In [1]:
#expt_list = [511510699, 511510664, 511510797, 511507650, 511510917,
#             511510675, 511510911, 511510860, 511510658, 511498500]
container_id = 511510699
session_idx = 1

In [2]:
drive_path = '/Volumes/Brain2016/'  ######## note... *changed to fix problem -- may need to remove ' 1'

import numpy as np
import pandas as pd
import os
import sys

import matplotlib.pyplot as plt
%matplotlib inline

/Users/philipmardoum/Desktop/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

manifest_path = os.path.join(drive_path,'BrainObservatory/manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)

expt_cont_list = boc.get_experiment_containers()

In [4]:
expt_session_info = pd.DataFrame(boc.get_ophys_experiments(experiment_container_ids=[container_id]))
#print(boc.get_experiment_containers(ids = [container_id]))
print(boc.get_experiment_containers(ids=[container_id]))

[{'targeted_structure': u'VISp', 'imaging_depth': 275, 'age_days': 124.0, 'id': 511510699, 'cre_line': u'Cux2-CreERT2'}]


In [5]:
# Create list of 3 session IDs in exp container, in standard order.
container_session_ids = [expt_session_info[expt_session_info['session_type'] == 'three_session_A']['id'].values[0],
                         expt_session_info[expt_session_info['session_type'] == 'three_session_B']['id'].values[0],
                         expt_session_info[expt_session_info['session_type'] == 'three_session_C']['id'].values[0]]

In [6]:
data_sets = []
specimens_by_session = []
#specimen_indices_by_session = []  # useless because this just ends up being a list of consec numbers
for i in range(3):
    data_sets.append(boc.get_ophys_experiment_data(ophys_experiment_id = container_session_ids[i]))
    specimens_by_session.append(data_sets[i].get_cell_specimen_ids())
    #specimen_indices_by_session.append(data_sets[i].get_cell_specimen_indices(list(specimens_by_session[i])))
specimens_by_session = np.array(specimens_by_session)
#specimen_indices_by_session = np.array(specimen_indices_by_session)

stable_specimen_ids = set(specimens_by_session[0]) & set(specimens_by_session[1]) & set(specimens_by_session[2])
stable_specimen_ids = np.array(list(stable_specimen_ids))

print(specimens_by_session[0].shape)
print(specimens_by_session[1].shape)
print(specimens_by_session[2].shape)

for i in range(3):
    print(data_sets[i].list_stimuli())

(319,)
(289,)
(243,)
[u'drifting_gratings', u'natural_movie_one', u'natural_movie_three', u'spontaneous']
[u'natural_movie_one', u'natural_scenes', u'spontaneous', u'static_gratings']
[u'locally_sparse_noise', u'natural_movie_one', u'natural_movie_two', u'spontaneous']


In [7]:
# After this point many variables are specific to the session chosen above
current_data_set = data_sets[session_idx]

# stim_tables = []
# for i in range(len(current_data_set.list_stimuli())):
#     stim_tables.append(current_data_set.get_stimulus_table(current_data_set.list_stimuli()[i]))
# print(current_data_set.list_stimuli()[2])
# temp = pd.DataFrame(stim_tables[2])
# temp

In [8]:
# stim_table = current_data_set.get_stimulus_table('natural_movie_one')
# stim_table

# Note... stim_table_m1 = current_data_set.get_stimulus_table('natural_movie_one') gives a start and end for each frame

In [9]:
# %matplotlib notebook
# plt.plot(stim_table.start.values)

In [10]:
# print(stim_table.start[18000])
# print(stim_table.end.values[-1])

NameError: name 'stim_table' is not defined

In [11]:
# # Takes way too long to run...
# from allensdk.brain_observatory.natural_scenes import NaturalScenes
# ns = NaturalScenes(current_data_set).

KeyboardInterrupt: 

In [ ]:
timestamps, traces = current_data_set.get_corrected_fluorescence_traces()

stable_specimen_indices = current_data_set.get_cell_specimen_indices(stable_specimen_ids)

traces_subset = traces[stable_specimen_indices, :]

In [ ]:
responses = traces[:,stim_table.start[0] : stim_table.end[0]]
responses.shape

In [ ]:
from allensdk.brain_observatory.natural_movie import NaturalMovie
from allensdk.brain_observatory import stimulus_info

nm1 = NaturalMovie(current_data_set, movie_name=stimulus_info.NATURAL_MOVIE_ONE)
sweep_response_nm1 = nm1.sweep_response

print(sweep_response_nm1.shape)

In [ ]:
# preallocate activity matrix: zeros(# cells, length of trace)
responses_movie = np.zeros((sweep_response_nm1.shape[1], len(sweep_response_nm1.iloc[0,0])))
for i in range(sweep_response_nm1.shape[1]):
    responses_movie[i,:] = sweep_response_nm1.iloc[0,i]

responses_movie.shape

In [ ]:
# for the cell we've been dealing with:
plt.figure(figsize=(8,6))
for i in range(100):
    plt.plot(sweep_response_nm1[str(i)].iloc[0])

In [ ]:
nm3 = NaturalMovie(current_data_set, movie_name=stimulus_info.NATURAL_MOVIE_THREE)
sweep_response_nm3 = nm3.sweep_response

sweep_response_nm3.shape

In [ ]:
sweep_response_nm3

In [ ]:
# preallocate activity matrix: zeros(# cells, length of trace)
responses_movie = np.zeros((sweep_response_nm3.shape[1], len(sweep_response_nm3.iloc[0,0])))
for i in range(sweep_response_nm3.shape[1]):
    responses_movie[i,:] = sweep_response_nm3.iloc[0,i]

responses_movie.shape

In [ ]:
plt.figure(figsize=(8,6))
for i in range(100):
    plt.plot(sweep_response_nm3[str(i)].iloc[0])